In [3]:
import networkx as nx
from sklearn.cluster import KMeans
from urllib import request

In [ ]:
url = "http://perso.telecom-paristech.fr/~bonald/graphs/"
dataset = "wikipedia_schools_undirected.graphml.gz"
download = request.urlretrieve(url + dataset, dataset)

In [ ]:
graph = nx.read_graphml(dataset, node_type=int)
print(nx.info(graph))

In [ ]:
n_nodes = graph.number_of_nodes()
degrees = np.array([graph.degrees(u) for u in graph.nodes()])

In [ ]:
adjacency = nx.to_scipy_sparse_array(graph)

### Embedding

In [ ]:
spectral = SpectralEmbedding()
weighted_spectral = SpectralEmbedding(node_weights="degree")

spectral.fit(adjacency)
weighted_spectral.fit(adjacency)

In [ ]:
def get_shift_embedding(embedding_, index=None, weighting="degree"):
    n_nodes, dim=np.shape(embedding_)
    if index is None:
        index = range(n_nodes)
    if type(weighting) == str:
        if weighting == "degree":
            weights_ = np.array([graph.degree(u) for u in index])
        elif weighting == "unitary":
            weights_ = np.ones(len(index))
    else:
        weights_ = weighting[index]
    centroid_ = np.sum(embedding_[index].T*weights_, axis=1).reshape(1, dim)/np.sum(weights_)
    return embedding_, -np.ones((n_nodes, 1)).dot(centroid_)

In [ ]:
embedding = spectral.embedding_
weighted_embedding = weighted_spectral.embedding_

In [ ]:
spectrum = spectral.eigenvalues_
weighted_spectrum = weighted_spectral.eigenvalues_

In [ ]:
shifted_embedding = get_shifted_embedding(embedding)

In [ ]:
np.shape(embedding)

### Global clustering

In [ ]:
n_clusters = 20
kmeans = kMeans(n_clusters, n_init=100)

In [ ]:
def sort_nodes(embedding, clusters, ranking, weighting = "degree"):
    index = np.array(cluster)
    if ranking == "degree":
        return index[np.argsort(- degrees[index])]
    elif ranking == "distance":
        local_distances =  np.linalg.norm(get_shift_embedding(embedding, index, weighting), axis=1)
        return index[np.argsort(local_distances[index])]
    elif ranking=="mixed":
        local_distances = np.linalg.norm(get_shift_embedding(embedding, index, weighting), axis=1)
        median = len(cluster)//2
        index1 = np.argsort(local_distances[index])
        index1[:median] = index[np.argsort(-degrees[index[index1[:median]]])]
        return index[index1]

In [ ]:
def get_clusters(embedding, normalize=True, ranking="mixed", weighting="degree", selection=None):
    
    if normalize:
        embedding_ = (embedding.T / np.linalg.norm(embedding, axis=1)).T
    else:
        embedding_ = embedding
    kmeans.fit(embedding_)
    labels = list(kmeans.labels_)
    clusters = [[] for k in range(n_clusters)]
    for u in range(len(labels)):
        clusters[labels[u]].append(u)
    if selection is not None:
        clusters = [list(set(c) & set(selection)) for c in clusters]
    clusters = sorted(clusters, key = len, reverse = True)
    return [sort_nodes(embedding, c, ranking, weighting) for c in clusters if len(c)>0]

In [ ]:
def show_top_nodes(clusters, k_clusters=20, k_nodes = 5):
    to_print = ""
    for i, c in enumerate(clusters):
        if i < k_clusters:
            to_print += str(i+1)+" (" + str(len(c)) + ") "
            for u in c[:k_nodes]:
                to_print += graph.node[u]["name"] + ","
            to_print += "\n"
    print(to_print)

In [ ]:
clusters = get_clusters(embedding, weighting="unitary")
shifted_clusters = get_clusters(shifted_embedding)
weighted_clusters = get_clusters(weighted_embedding)

In [ ]:
show_top_nodes(clusters)
show_top_nodes(shifted_clusters)
show_top_nodes(weighted_clusters)

### Selective clustering

In [ ]:
def get_nodes_by_category(category):
    return [u for u in graph.nodes() if "category" in graph.node[u] and category in graph.node[u]["category"]]

In [ ]:
people = get_nodes_by_category("People.")
len(people)

In [ ]:
selected_nodes = people
subgraph = nx.Graph(graph.subgraph(people))
nx.is_connected(subgraph)

In [ ]:
factor = 10
factors = np.array([1+(factor - 1)* int(u in selected_nodes) for u in graph.nodes()])
uniform_weights = factors
degree_weights = degrees*factors

In [ ]:
selective_spectral = SpectralEmbedding(node_weights = uniform_weights)
selective_weighted_spectral = SpectralEmbedding(node_weights = degree_weights)


selective_spectral.fit(adjacency)
selective_weighted_spectral.fit(adjacency)

In [ ]:
selective_embedding = selective_sepectral.embedding_
selective_weighted_embedding = selective_weighted_spectral.embedding_
selective_shifted_embedding = selective_

In [ ]:
selective_clusters = get_clusters(
    selective_embedding,
    ranking="degree",
    selection = selected_nodes
)



selective_shifted_clusters = get_clusters(
    selective_shifted_embedding, 
    ranking="degree",
    selection = selected_nodes
)



selective_weighted_clusters = get_clusters(
    selective_weighted_embedding, 
    ranking="degree", 
    selction=selected_nodes
)

In [ ]:
show_top_nodes(selective_clusters)
show_top_nodes(selective_shifted_clusters)
show_top_nodes(selective_weighted_clusters)